In [1]:
from langgraph.graph import Graph

In [3]:
def function_1(input):
    return input + " from function_1"

def function_2(input):
    return input + " to function_2"

In [8]:
workflow = Graph()

workflow.add_node("node_1",function_1)
workflow.add_node("node_2",function_2)

workflow.add_edge('node_1','node_2')
workflow.set_entry_point('node_1')
workflow.set_finish_point('node_2')
app = workflow.compile()

In [6]:
app.invoke('I am going')

'I am going from function_1 to function_2'

In [10]:
for output in app.stream("I am going"):
    for key,value in output.items():
        print("key:",key)
        print(value)

key: node_1
I am going from function_1
key: node_2
I am going from function_1 to function_2


LLM call using Langgraph

In [3]:
from config import LLAMA_MODEL
from langchain_ollama import ChatOllama

In [4]:
model = ChatOllama(model="llama3.2")

In [17]:
workflow = Graph()

def llm_function_1(input):
    prompt = "Your task is to get the only country name from the sentence: "+ input
    value = model.invoke(prompt)
    return value.content

def llm_function_2(input):
    return "Country name is "+input

In [18]:
workflow.add_node('node_1',llm_function_1)
workflow.add_node('node_2',llm_function_2)

workflow.add_edge('node_1','node_2')
workflow.set_entry_point('node_1')
workflow.set_finish_point('node_2')

app=workflow.compile()

In [19]:
app.invoke("My favurite country is India")

'Country name is The country name is:\n\nIndia'

RAG Approach

In [47]:
from langchain_community.vectorstores import FAISS
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [24]:
def rag_function_1(state):
    val = state['messages']
    prompt = "Your task is to get the only specific topic name from the sentence: "+ val[0]
    value = model.invoke(prompt)
    state['messages'].append(value.content)
    return state

def rag_function_2(state):
    retriever = FAISS.load_local('web_data',OllamaEmbeddings(model=LLAMA_MODEL),allow_dangerous_deserialization=True).as_retriever()
    prompt = '''Your task is to give the answer from the context based on given question:
            {context}

            Question : {question}
        '''
    prompt = ChatPromptTemplate.from_template(prompt)
    chain = (
        {"context":retriever,"question":RunnablePassthrough()}
        |prompt
        |model
    )

    que = state['messages']
    value =chain.invoke(que[0])
    return value


In [25]:
workflow = Graph()

workflow.add_edge('node_1','node_2')
workflow.add_node('node_1',rag_function_1)
workflow.add_node('node_2',rag_function_2)


workflow.set_entry_point('node_1')
workflow.set_finish_point('node_2')

app = workflow.compile()

In [26]:
input = {"messages":["What is langsmith?"]}
for output in app.stream(input):
    for key,value in output.items():
        print("Key:",key)
        print("Value:",value)

Key: node_1
Value: {'messages': ['What is langsmith?', 'The specific topic name mentioned in the sentence "What is Langsmith?" is:\n\n* Langsmith']}
Key: node_2
Value: content='Based on the given context, LangSmith is a platform for building production-grade LLM (Large Language Model) applications. This information can be found in the first and third documents provided. The description of LangSmith is "LangSmith is a platform for building production-grade LLM applications."' additional_kwargs={} response_metadata={'model': 'llama3', 'created_at': '2024-12-21T05:14:13.1456728Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 443259248400, 'load_duration': 63968900, 'prompt_eval_count': 824, 'prompt_eval_duration': 400826446000, 'eval_count': 58, 'eval_duration': 42360245000} id='run-f0cef1aa-d22c-44ab-a4ce-85a362ba4004-0' usage_metadata={'input_tokens': 824, 'output_tokens': 58, 'total_tokens': 882}


In [27]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate

In [84]:
class TopicSelectionParser(BaseModel):
    Topic: str = Field(description="Selected Topic")

parser = PydanticOutputParser(pydantic_object=TopicSelectionParser)

In [85]:
prompt = '''Your task is to get the only specific topic name from the sentence: {question}  {format_instruction}'''

template = PromptTemplate(template=prompt,input_variables=['question'],partial_variables={"format_instruction":parser.get_format_instructions()})


chain = template |model | parser

In [86]:
val =chain.invoke({'question':'What is langsmith?','format_instruction':parser.get_format_instructions()})

In [87]:
val

TopicSelectionParser(Topic='langsmith')

Conditional State Graph

In [44]:
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel,Field

In [81]:
class OutputParser(BaseModel):
    Topic: str = Field(description="Selected Topic")

parser = PydanticOutputParser(pydantic_object=OutputParser)

In [144]:
def function_1(state):
    prompt = '''Your task is to get the only specific topic name from the sentence: {query}  {format_instruction}'''

    prompt = PromptTemplate(template=prompt,input_variables=['query'],partial_variables={"format_instruction":parser.get_format_instructions()})
    chain = prompt | model | parser
    
    message=state['messages']
    query = message[-1]
    print("queyr:",query)
    value = chain.invoke({'query':query,"format_instruction":parser.get_format_instructions()})
    print("Value:",value)
    state['messages'].append(value.Topic)
    print("state:",state)
    return state

def function_2(state):
    print("RAG")
    retriever = FAISS.load_local('web_data',OllamaEmbeddings(model=LLAMA_MODEL),allow_dangerous_deserialization=True).as_retriever()
    prompt = '''Your task is to answer the question based on provided context:
             {context}
             
             Question : {question}
    '''
    prompt = ChatPromptTemplate.from_template(template=prompt)
    chain = (
        {"context":retriever,"question":RunnablePassthrough()}
        |prompt
        | ChatOllama(model=LLAMA_MODEL)
    )

    que = state['messages']
    print("QUE:",que[0])
    que = que[0]
    value = chain.invoke(que)
    return value

def function_3(state):
    prompt = '''Your task is to answer the following question: {question}'''
    prompt = PromptTemplate(template=prompt)
    chain=prompt | model
    que = state['messages']
    value =chain.invoke({'question':que[0]})
    print("Answer LLM:",value.content)
    return value

def router(state):
    messages = state['messages']
    messages = messages[-1]
    print("Reouter:",messages)
    if 'Langsmith' in messages:
        return "RAG"
    else:
        return "LLM"


In [62]:
from typing import TypedDict,Annotated,Sequence
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
    messages : Annotated[Sequence[BaseMessage],operator.add]


In [146]:
from langgraph.graph import StateGraph,END

workflow = StateGraph(AgentState)

workflow.add_node('agent',function_1)
workflow.add_node('RAG',function_2)
workflow.add_node('LLM',function_3)

workflow.set_entry_point('agent')
workflow.add_conditional_edges('agent',router,{
    "LLM":"LLM",
    "RAG":"RAG"
})

workflow.add_edge('LLM',END)
workflow.add_edge('RAG',END)

app = workflow.compile()




In [147]:
state = {'messages':['what is Langsmith?']}
app.invoke(state)
# for output in app.stream():
#     for key,value in output.items():
#         print("Key:",key)
#         print('Value:',value)

queyr: what is Langsmith?
Value: Topic='Langsmith'
state: {'messages': ['what is Langsmith?', 'Langsmith']}
Reouter: Langsmith
RAG
QUE: what is Langsmith?


{'messages': ['what is Langsmith?',
  'Langsmith',
  'what is Langsmith?',
  'Langsmith']}

Tools Making and Calling

In [40]:
from langchain.tools import tool
from config import LLAMA_MODEL
from langchain_ollama import ChatOllama

In [42]:
@tool
def multiply(a:int,b:int):
    "Multiply two numbers together"
    return a * b

In [43]:
model = ChatOllama(model="llama3.2")


In [44]:
tools = [multiply]
model = model.bind_tools(tools=tools)

In [45]:
response = model.invoke("What is value of 35 * 2 ?")

In [180]:
response.response_metadata.get('message')


{'role': 'assistant',
 'content': '',
 'tool_calls': [{'function': {'name': 'multiply',
    'arguments': {'a': 35, 'b': 2}}}]}

In [95]:
from langgraph.graph import StateGraph,END
import json

In [253]:
@tool
def multiply_tools(a:int,b:int)->int:
    "Multiply two numbers together"
    return a * b

model = ChatOllama(model="llama3.2")
tools = [multiply_tools]
model = model.bind_tools(tools=tools)

In [254]:
def invoke_model(state):
    message = state.get('messages')
    message = message[-1]
    
    response = model.invoke(message)
    print(response)
    state['messages'].append(response.response_metadata.get('message').get('tool_calls')[0])

    return state

def invoke_tools(state):
    #print("State:",state)
    arg = state.get('messages')[1].get('function').get('arguments')
    
    val = {}
    for key, value in arg.items():
        val[key]=int(value)
    
    response = multiply_tools.invoke(val)
    state['messages'].append({"value":response})
    return {"value":[response]}

def router(state):
    message = state.get('messages')[1].get('function').get('name')
    if message=='multiply_tools':
        return 'tools'
    else:
        return 'else'
    return state

In [255]:
graph = StateGraph(AgentState)

graph.add_node('agent',invoke_model)
graph.add_node('tools',invoke_tools)


graph.set_entry_point('agent')
graph.add_conditional_edges('agent',router,{"tools":"tools","else":END})
graph.add_edge('tools',END)

app = graph.compile()

In [258]:
messages = {'messages':['what is 35+2 ?']}
ans = app.invoke(messages)

content='' response_metadata={'model': 'llama3.2', 'created_at': '2024-12-22T17:28:01.0214053Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'multiply_tools', 'arguments': {'a': '35', 'b': '37'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 9833989400, 'load_duration': 130417000, 'prompt_eval_count': 168, 'prompt_eval_duration': 1882000000, 'eval_count': 23, 'eval_duration': 7818000000} id='run-e6a6d338-e8b4-4584-9652-222b1c4afc1d-0' tool_calls=[{'name': 'multiply_tools', 'args': {'a': '35', 'b': '37'}, 'id': '915c70c9-901e-493b-b03c-6a27f63b142a', 'type': 'tool_call'}] usage_metadata={'input_tokens': 168, 'output_tokens': 23, 'total_tokens': 191}


In [259]:
ans

{'messages': ['what is 35+2 ?',
  {'function': {'name': 'multiply_tools',
    'arguments': {'a': '35', 'b': '37'}}},
  'what is 35+2 ?',
  {'function': {'name': 'multiply_tools',
    'arguments': {'a': '35', 'b': '37'}}},
  {'value': 1295}]}

In [237]:
v=multiply_tools.invoke({'a':10,'b':2})

In [238]:
v

20